<a href="https://colab.research.google.com/github/divyanshuraj6815/END-NLP/blob/main/experiment_7/Sentiment_Analysis_using_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [1]:
!wget -N http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
!unzip *.zip

--2020-12-10 10:59:38--  http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip [following]
--2020-12-10 10:59:38--  https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6372817 (6.1M) [application/zip]
Saving to: ‘stanfordSentimentTreebank.zip’

stanfordSentimentTr 100%[===================>]   6.08M  4.48MB/s    in 1.4s    

2020-12-10 10:59:40 (4.48 MB/s) - ‘stanfordSentimentTreebank.zip’ saved [6372817/6372817]

Archive:  stanfordSentimentTreebank.zip
   creating: stanfordSentimentTreebank/
  inflating: stanfordSentimentTreebank/datasetSentences.txt  
   creating: 

In [2]:
input_file = '/content/stanfordSentimentTreebank/datasetSentences.txt'
output_file = '/content/stanfordSentimentTreebank/sentiment_labels.txt'

In [3]:
!head /content/stanfordSentimentTreebank/datasetSentences.txt

sentence_index	sentence
1	The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
2	The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth .
3	Effective but too-tepid biopic
4	If you sometimes like to go to the movies to have fun , Wasabi is a good place to start .
5	Emerges as something rare , an issue movie that 's so honest and keenly observed that it does n't feel like one .
6	The film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .
7	Offers that rare combination of entertainment and education .
8	Perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .
9	Ste

In [4]:
import pandas as pd
input_pd = pd.read_csv (input_file, sep = '\t', index_col = 'sentence_index')
input_pd.head ()

,sentence
sentence_index,
1,The Rock is destined to be the 21st Century 's...
2,The gorgeously elaborate continuation of `` Th...
3,Effective but too-tepid biopic
4,If you sometimes like to go to the movies to h...
5,"Emerges as something rare , an issue movie tha..."


In [5]:
!head /content/stanfordSentimentTreebank/sentiment_labels.txt

phrase ids|sentiment values
0|0.5
1|0.5
2|0.44444
3|0.5
4|0.42708
5|0.375
6|0.41667
7|0.54167
8|0.33333


In [6]:
output_pd = pd.read_csv (output_file, sep = '|', index_col='phrase ids')
for i, row in output_pd.iterrows ():
  x = row['sentiment values']
  if x >= 0.4 and x < 0.6:
    x = 1
  elif x <= 0.4:
    x = 0
  else:
    x = 2
  output_pd.at[i, 'sentiment values'] = x
output_pd.head ()

,sentiment values
phrase ids,
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [7]:
print (input_pd['sentence'].count (), output_pd['sentiment values'].count ())

11855 239232


In [8]:
output_pd['sentiment values'].min (), output_pd['sentiment values'].max ()

(0.0, 2.0)

In [9]:
df = pd.concat ([input_pd, output_pd.reindex (input_pd.index)], axis = 1)
df = df.rename (columns = {'sentence' : 'tweets', 'sentiment values' : 'labels'})

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [10]:
#import pandas as pd
#df = pd.read_csv('drive/MyDrive/END Course Files/Lecture 6/tweets.csv')
df.head()

,tweets,labels
sentence_index,,
1,The Rock is destined to be the 21st Century 's...,1.0
2,The gorgeously elaborate continuation of `` Th...,1.0
3,Effective but too-tepid biopic,1.0
4,If you sometimes like to go to the movies to h...,1.0
5,"Emerges as something rare , an issue movie tha...",0.0


In [11]:
df.shape

(11855, 2)

In [12]:
df.labels.value_counts()

1.0    6240
0.0    2813
2.0    2802
Name: labels, dtype: int64

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [15]:
!sudo pip install google_trans_new

In [23]:
translated = pd.DataFrame (columns = ['tweets', 'labels'])

In [ ]:
import random
import google_trans_new
from google_trans_new import google_translator

translator = google_translator()
print (translator)

for index, element in df.iterrows ():
  if index < 5000:
    continue
  sentence = element['tweets']
  available_langs = list(google_trans_new.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {google_trans_new.LANGUAGES[trans_lang]}")

  translations = translator.translate(sentence, lang_tgt=trans_lang) 

  #t_text = [t for t in translations]
  t_text = translations
  #print(t_text)

  translations_en_random = translator.translate(t_text, lang_src=trans_lang, lang_tgt='en') 
  #en_text = [t.text for t in translations_en_random]
  en_text = translations_en_random
  #print(en_text)
  translated.loc[index] = [en_text, element['labels']]
  if index % 100 == 0:
    print (index)

In [64]:
translated.head ()

,tweets,labels
1,Rock is set to be the 21st century new `` Cona...,1.0
2,The superbly crafted sequel to The Lord of the...,1.0
3,Effective but excellent biopers,1.0
4,"If you ever like to go to the movies, Wasabi i...",1.0
5,"It emerges as an uncommon thing, an release fi...",0.0


In [29]:
translated.to_csv ('translated.csv')

In [67]:
for index, element in translated.iterrows ():
  print (index, element)
  break

1 tweets    Rock is set to be the 21st century new `` Cona...
labels                                                    1
Name: 1, dtype: object


In [30]:
print (translated['tweets'].count ())

7617


In [58]:
df = df.append (translated, ignore_index=True)

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [59]:
def random_deletion(words, p=0.3):
    words = words.split (' ')
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

In [76]:
r_del = pd.DataFrame (columns = ['tweets', 'labels'])

In [77]:
for index, element in df.iterrows ():
  if type (element['tweets']) == type (list ()):
    print (index, element['tweets'])
    print (len (element))
    #print ('ele=>', element['tweets'][0])
    #print (element['tweets'][1])
    #print (element[1])
    df.at[index, 'tweets'] = element['tweets'][0]
    element['tweets'] = element['tweets'][0]
  r_del.loc[index] = [random_deletion (element['tweets']), element['labels']]
  #break

18948 ['The wiser souls would graciously pretend not to see him and leave him lying there.', 'The wiser souls would graciously pretend not to see her and leave her lying there.']
2


In [78]:
r_del.head ()

,tweets,labels
0,is be the 21st Century 's `` '' and he to make...,1.0
1,gorgeously continuation `` the Rings '' trilog...,1.0
2,Effective,1.0
3,If sometimes like to movies to have Wasabi is ...,1.0
4,Emerges as something an issue movie 's so hone...,0.0


In [79]:
df = df.append (r_del, ignore_index=True)

In [80]:
df.to_csv ('final_data.csv')

In [81]:
df['tweets'].count ()

38944

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [14]:
def random_swap(sentence, n=2): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [82]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [137]:
df = pd.read_csv ('final_data.csv')

indexes = []
for index, element in df.iterrows ():
  #print (element)
  try:
    if len (element['tweets']) == 0:
      df.at[index, 'tweets'] = 'not applicable'
      indexes.append (index)
  except:
      df.at[index, 'tweets'] = 'not applicable'
      indexes.append (index)

print (indexes)
#df = df.drop (indexes)

df.head ()

[31902, 32275, 32666, 33337, 34291, 34807, 34978, 35141, 36661, 37012, 37080, 37436, 38163]


,Unnamed: 0,tweets,labels
0,0,The Rock is destined to be the 21st Century 's...,1.0
1,1,The gorgeously elaborate continuation of `` Th...,1.0
2,2,Effective but too-tepid biopic,1.0
3,3,If you sometimes like to go to the movies to h...,1.0
4,4,"Emerges as something rare , an issue movie tha...",0.0


In [136]:
df.reindex ([i for i in range (38900)])

,Unnamed: 0,tweets,labels
0,0,The Rock is destined to be the 21st Century 's...,1.0
1,1,The gorgeously elaborate continuation of `` Th...,1.0
2,2,Effective but too-tepid biopic,1.0
3,3,If you sometimes like to go to the movies to h...,1.0
4,4,"Emerges as something rare , an issue movie tha...",0.0
...,...,...,...
38895,38895,Despite the aesthetic that the state of modern...,2.0
38896,38896,Madeline than,2.0
38897,38897,This an egotistical endeavor from the daughter...,2.0
38898,38898,"- very, very slow.",1.0


In [138]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [139]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [140]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(1, df.shape[0])] 

In [141]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

In [142]:
for element in twitterDataset:
  print (vars(element))
  break

{'tweets': ['The', 'gorgeously', 'elaborate', 'continuation', 'of', '`', '`', 'The', 'Lord', 'of', 'the', 'Rings', "''", 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'can', 'not', 'adequately', 'describe', 'co', '-', 'writer\\/director', 'Peter', 'Jackson', "'s", 'expanded', 'vision', 'of', 'J.R.R.', 'Tolkien', "'s", 'Middle', '-', 'earth', '.'], 'labels': 1.0}


Finally, we can split into training, testing, and validation sets by using the split() method:

In [143]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [144]:
(len(train), len(valid))

(33102, 5841)

An example from the dataset:

In [145]:
vars(train.examples[10])

{'labels': 1.0,
 'tweets': ['The',
  'premise',
  'kegger',
  'comedy',
  'sounded',
  'four',
  'six',
  '-',
  'packs',
  'and',
  'a',
  'pitcher',
  'margaritas',
  'in',
  'but',
  'the',
  'film',
  'been',
  'written',
  '...',
  'in',
  'the',
  'thrall',
  'of',
  'a',
  'hangover',
  '.']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [146]:
Tweet.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [147]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  22564
Size of label vocab :  3
Top 10 words appreared repeatedly : [('.', 26643), (',', 23581), ('the', 21247), ('and', 15425), ('of', 15065), ('a', 14768), ('to', 10033), ('is', 9375), ('-', 8366), ("'s", 6849)]
Labels :  defaultdict(<function _default_unk_index at 0x7f26f68750d0>, {1.0: 0, 2.0: 1, 0.0: 2})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [148]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [149]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [150]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [151]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [152]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [153]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(22564, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 7,011,103 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [154]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [155]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        #print (tweet, tweet_lengths)
        #break
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [156]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            #print (tweet, tweet_lengths)
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [157]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.027 | Train Acc: 52.31%
	 Val. Loss: 1.016 |  Val. Acc: 52.60% 

	Train Loss: 1.002 | Train Acc: 53.20%
	 Val. Loss: 0.999 |  Val. Acc: 53.85% 

	Train Loss: 0.949 | Train Acc: 59.09%
	 Val. Loss: 0.967 |  Val. Acc: 56.77% 

	Train Loss: 0.883 | Train Acc: 66.82%
	 Val. Loss: 0.943 |  Val. Acc: 59.33% 

	Train Loss: 0.824 | Train Acc: 73.41%
	 Val. Loss: 0.916 |  Val. Acc: 62.34% 

	Train Loss: 0.778 | Train Acc: 78.29%
	 Val. Loss: 0.896 |  Val. Acc: 64.86% 

	Train Loss: 0.744 | Train Acc: 81.68%
	 Val. Loss: 0.882 |  Val. Acc: 66.64% 

	Train Loss: 0.715 | Train Acc: 84.56%
	 Val. Loss: 0.864 |  Val. Acc: 68.44% 

	Train Loss: 0.694 | Train Acc: 86.64%
	 Val. Loss: 0.860 |  Val. Acc: 68.74% 

	Train Loss: 0.678 | Train Acc: 88.09%
	 Val. Loss: 0.860 |  Val. Acc: 68.49% 



## Model Testing

In [158]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 2:"Positive", 1:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [159]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Negative'